---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

In [75]:
def answer_one():
    spam = spam_data[spam_data['target']==1]['target'].count()
    all = spam_data['target'].count()
    return 100*spam/all
#answer_one()

In [76]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vectorizer = CountVectorizer()
    vectorized = vectorizer.fit(X_train)
    tokens = vectorized.get_feature_names()
    max_len=1
    token=''
    for word in tokens:
        if len(word)>max_len:
            max_len = len(word)
            token = word
    return  token


In [74]:
answer_two()

32

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [4]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vectorizer = CountVectorizer().fit(X_train)
    vector_train = vectorizer.transform(X_train)
    vector_test = vectorizer.transform(X_test)
    
    #fit the classifier:
    classifier_MNB = MultinomialNB()
    classifier_MNB.fit(vector_train, y_train)
    pred = classifier_MNB.predict(vector_test)
    
    return roc_auc_score(y_test,pred)

In [45]:
answer_three()

0.95813668234215565

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    #get highest and lowest TF-Idfs from X_Train
    vectorizer = TfidfVectorizer()
    vectorizer.fit(X_train)
    x_vect = vectorizer.transform(X_train)
    
    #features
    vectorizer = TfidfVectorizer()
    vectorizer.fit(X_train)
    x_vect = vectorizer.transform(X_train)

    #features
    tokens = np.array(vectorizer.get_feature_names())
    #tf-idf for each token
    tf_idf_per_token=x_vect.max(0).toarray()[0]
    ser = pd.Series(data=tf_idf_per_token, index=tokens)
    
    #has to be sorted twice since we have two-factor sorting being done.
    #first sort values then by alphabet (both asc)
    #then sort values desc and letters asc
    lowest = ser.iloc[np.lexsort([ser.index, ser.values])][0:20]
    highest = ser.iloc[np.lexsort([ser.index, -1*ser.values])][0:20]
    
    return lowest,highest

In [116]:
answer_four()

20

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [131]:
def answer_five():
    vectorizer = TfidfVectorizer(min_df=3)
    vectorizer.fit(X_train)
    train_vect = vectorizer.transform(X_train)
    test_vect = vectorizer.transform(X_test)
    
    #fit the classifier:
    classifier_MNB = MultinomialNB(alpha=.1)
    classifier_MNB.fit(train_vect, y_train)
    pred = classifier_MNB.predict(test_vect)
    return roc_auc_score(y_test,pred)

In [132]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [127]:
def answer_six():
    spam_copy = spam_data.copy()
    spam_copy['length'] = spam_data['text'].apply(len)
    spam_lengths = spam_copy.groupby('target')['length'].mean()
    return (spam_lengths.iloc[0],spam_lengths.iloc[1])

In [128]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [5]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [148]:
from sklearn.svm import SVC

def answer_seven():
    #first vectorize text
    vectorizer = TfidfVectorizer(min_df=5)
    vectorizer.fit(X_train)
    train_vect = vectorizer.transform(X_train)
    test_vect = vectorizer.transform(X_test)    
    
    #then count characters in train/test split X data
    x_train_chars = X_train.apply(len)
    x_test_chars = X_test.apply(len)
    
    #combine/inject new data with old
    x_train_aug = add_feature(train_vect, x_train_chars)
    x_test_aug = add_feature(test_vect,x_test_chars)
    
    #train SCV
    model = SVC(C=10000).fit(x_train_aug, y_train)
    pred = model.predict(x_test_aug)
    return roc_auc_score(y_test,pred)

In [149]:
answer_seven()

0.9852307872118532

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [156]:
def answer_eight():
    #quick helper function to strip all non-digits from a string and get length
    def get_only_nums_length(some_string):
        return len(''.join(c for c in some_string if c.isdigit()))
    
    spam_copy = spam_data.copy()
    spam_copy['length'] = spam_data['text'].apply(get_only_nums_length)
    spam_lengths = spam_copy.groupby('target')['length'].mean()
    return (spam_lengths.iloc[0],spam_lengths.iloc[1])

In [157]:
answer_eight()

(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    def get_only_nums_length(some_string):
        return len(''.join(c for c in some_string if c.isdigit()))
    
    #first vectorize text
    vectorizer = TfidfVectorizer(min_df=5)
    vectorizer.fit(X_train)
    train_vect = vectorizer.transform(X_train)
    test_vect = vectorizer.transform(X_test)    
    
    #then count characters in train/test split X data
    x_train_chars = X_train.apply(len)
    x_test_chars = X_test.apply(len)
    x_train_nums = X_train.apply(get_only_nums_length)
    x_test_nums = X_test.apply(get_only_nums_length)
    
    #combine/inject new data with old
    x_train_aug = add_feature(train_vect, x_train_chars)
    x_test_aug = add_feature(test_vect,x_test_chars)
    x_train_aug = add_feature(x_train_aug, x_train_nums)
    x_test_aug = add_feature(x_test_aug, x_test_nums)
    
    #train regression
    model=LogisticRegression(C=100).fit(x_train_aug,y_train)
    pred = model.predict(x_test_aug)
    
    return roc_auc_score(y_test,pred)

In [13]:
answer_nine()

0.9793015131848315

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [42]:
def answer_ten():
    import re
    spam_copy = spam_data.copy()
    spam_copy['num_special'] = spam_data['text'].apply(
        lambda x: len(re.sub(r'\w','', x)))
    avgs = spam_copy.groupby('target')['num_special'].mean()
    return (float(avgs.iloc[0]),float(avgs.iloc[1]))

In [43]:
answer_ten()
#(17.29181347150259, 29.041499330655956)

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [15]:
import re
def answer_eleven():
    def get_only_nums_length(some_string):
        return len(''.join(c for c in some_string if c.isdigit()))
    
    #first vectorize text
    vectorizer = CountVectorizer(min_df=5, ngram_range=(2, 5), analyzer='char_wb').fit(X_train)
    train_vect = vectorizer.transform(X_train)
    test_vect = vectorizer.transform(X_test)
    feature_names = vectorizer.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count']
    
    #then count characters in train/test split X data
    x_test_chars = X_test.apply(len)
    x_test_nums = X_test.apply(get_only_nums_length)
    x_test_spec = X_test.apply(lambda x: len(re.sub(r'\w','', x)))
    x_train_chars = X_train.apply(len)
    x_train_nums = X_train.apply(get_only_nums_length)
    x_train_spec = X_train.apply(lambda x: len(re.sub(r'\w','', x)))
    
    #combine/inject new data with old
    x_test_aug = add_feature(test_vect,x_test_chars)
    x_test_aug = add_feature(x_test_aug, x_test_nums)
    x_test_aug = add_feature(x_test_aug, x_test_spec)
    x_train_aug = add_feature(train_vect, x_train_chars)
    x_train_aug = add_feature(x_train_aug, x_train_nums)
    x_train_aug = add_feature(x_train_aug, x_train_spec)

    
    #train regression, collect AUC, and sort features based on importance
    model=LogisticRegression(C=100).fit(x_train_aug,y_train)
    pred = model.predict(x_test_aug)  
    auc = roc_auc_score(y_test,pred)
    coeffs = model.coef_[0]
    some_data = pd.DataFrame(list(zip(coeffs.tolist(), feature_names)), 
        columns=['coeff','feature'])
    some_data['coeff_abs'] = some_data['coeff'].apply(abs)
    some_data=some_data.sort_values(['coeff','feature']).reset_index()
    smallest = list(some_data.iloc[0:10]['feature'])
    largest = list(reversed(list(some_data.iloc[-10:]['feature'])))
    return (auc, smallest, largest)

In [16]:
answer_eleven()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])